In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import lightgbm as lgb
from sklearn.cross_validation import StratifiedKFold
from tqdm import tqdm
import gc
import time

/home/ashu/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
print('loading files...')
train = pd.read_csv('ptrain.csv')
test = pd.read_csv('ptest.csv')
sample_submission = pd.read_csv('sample_submission.csv')

loading files...


In [3]:
train.head()

,attend,meet_john,you_are,request,sure,stand,remind,pvr,pay,by_2011,...,in,you,to,is,on,at,your,for,word_count,label
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,6,ham
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.070389,0.000000,0.05499,0.000000,0.00000,0.000000,0.000000,0.000000,10,info
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.060334,0.053006,0.00000,0.000000,0.03474,0.033967,0.030581,0.066897,16,info
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.337869,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,11,ham
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.00000,0.631649,0.00000,0.000000,0.000000,0.000000,4,ham


In [4]:
test.head()

,attend,meet_john,you_are,request,sure,stand,remind,pvr,pay,by_2011,...,in,you,to,is,on,at,your,for,word_count,RecordNo
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.31703,0.000000,0.000000,5.0,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.026815,0.0,0.020949,0.000000,0.000000,0.00000,0.013591,0.000000,40.0,2
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,3.0,3
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.073450,0.0,0.057381,0.000000,0.000000,0.00000,0.000000,0.000000,12.0,4
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.077345,0.019851,0.00000,0.034949,0.019113,26.0,5


In [5]:
sample_submission.head()

,RecordNo,Label
0,1,ham
1,2,ham
2,3,ham
3,4,ham
4,5,ham


In [6]:
target = train['label'].astype('category')
target = target.cat.codes
test_ID = test['RecordNo']
train.drop(['label'] , axis=1 , inplace = True)
test.drop(['RecordNo'] , axis=1 , inplace = True)

In [8]:
# lgb

t1 = time.time()

X = train
X = X.values
y = target.values


feat_imp = []
nrounds = 5000 
kfold = 5

params = { 'metric':'multi_logloss','learning_rate' : 0.01, 'max_depth':10, 'max_bin':15,  'objective': 'multiclass', 
          'num_class':3, 'feature_fraction': 0.8,'bagging_fraction':0.9,'bagging_freq':10,  'min_data': 500}

skf = StratifiedKFold(n_splits=kfold, random_state=1)
for i, (train_index, test_index) in enumerate(skf.split(X, y)):
    print(' lgb kfold: {}  of  {} : '.format(i+1, kfold))
    X_train, X_eval = X[train_index], X[test_index]
    y_train, y_eval = y[train_index], y[test_index]
    lgb_model = lgb.train(params, lgb.Dataset(X_train, label=y_train), nrounds, lgb.Dataset(X_eval, label=y_eval), 
                          verbose_eval=10, early_stopping_rounds = 100)
   
    l = lgb_model.predict(test.values, num_iteration=lgb_model.best_iteration)
    
    sub += l/(kfold)
    feat_imp.append(lgb_model.feature_importance())


    
t2 = time.time()
print "Time taken {} hr".format((t2-t1)/(60*60))

TypeError: __init__() takes at least 2 arguments (3 given)

In [ ]:
s = np.zeros([test.shape[0]])
for i in tqdm(range(test.shape[0])) :
    s[i] = sub[i].argmax()

In [ ]:
submission=test_ID.to_frame()
submission['RecordNo'] = s
submission.to_csv('lgb_1.csv', index=False, float_format='%.5f')